In [5]:
import os
import requests
from dotenv import load_dotenv 
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
import openai
import ollama

In [6]:
OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2"

In [7]:
messages = [
    {"role": "user", "content": "what is 2+2?"}
]

In [8]:
response = ollama.chat(model=MODEL, messages=messages)
print(response['message']['content'])

2 + 2 = 4


In [9]:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [10]:
system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [11]:
def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [12]:
def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [15]:
def summarize(url):
    website = Website(url)
    response = ollama.chat(
        model=MODEL, 
        messages=messages_for(website))
    return response['message']['content']

In [16]:
summarize("https://edwarddonner.com")

"**Summary of the Website**\n==========================\n\n*   The website is run by Edward Donner, a co-founder and CTO of Nebula.io, an AI company applying AI to help people discover their potential.\n*   The main focus of the website appears to be showcasing the capabilities of Nebula.io's proprietary LLMs (Large Language Models) in talent sourcing and management.\n*   There are resources available for learning about mastering AI and LLM engineering, transitioning from a software engineer to an AI data scientist, and using LLMs in various ways.\n\n**News and Announcements**\n-------------------------\n\n*   **Mastering AI and LLM Engineering - Resources**: Published on November 13, 2024.\n*   **From Software Engineer to AI Data Scientist – resources**: Published on October 16, 2024.\n*   **Outsmart LLM Arena – a battle of diplomacy and deviousness**: Published on June 26, 2024.\n*   **Choosing the Right LLM: Toolkit and Resources**: Published in August 2024 (exact date not specified

In [17]:
def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [18]:
display_summary("https://edwarddonner.com")

**Summary**
================

* The website is run by Ed, a co-founder and CTO of Nebula.io, an AI startup that applies AI to help people discover their potential.
* Nebla.io uses proprietary LLMs for talent matching and has patented its matching model.
* The website features:
	+ An "Outsmart" arena where LLMs compete in diplomacy and deception challenges.
	+ Resources and articles on mastering AI, LLM engineering, software engineer to data scientist career paths, and choosing the right LLM.

**News/Announcements**
======================

* **Mastering AI and LLM Engineering – Resources**: Published November 13, 2024
* **From Software Engineer to AI Data Scientist – resources**: Published October 16, 2024
* **Outsmart LLM Arena – a battle of diplomacy and deviousness**: Published June 26, 2024

In [19]:
display_summary("https://en.wikipedia.org/wiki/Main_Page")

Did you know that:

* This article has over 122 million characters in its edit history!
* There are over 1,000 languages available on Wikipedia, including many minority languages.
* The website's URL has changed over 100 times since its inception in 2001.

Let me know if you have any other questions about Wikipedia!